In [1]:
!pip3 install kafka-python

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [84]:
from kafka import KafkaProducer, KafkaConsumer
from kafka.admin import KafkaAdminClient, NewTopic
import json

In [85]:
def read_dta_from_file(file_path):
    with open(file_path, "r") as file:
        data = json.load(file)
    return data

In [86]:
def create_topics(topics, bootstrap_servers = 'localhost:9092'):
    admin_client = KafkaAdminClient(bootstrap_servers=bootstrap_servers)
    topic_list = [ NewTopic(topic,num_partitions=1, replication_factor=1) for topic in topics ]
    admin_client.create_topics(new_topics=topic_list, validate_only=False)

In [87]:
def producer_to_topics(data, topics, bootstrap_servers = 'localhost:9092'):
    producer = KafkaProducer(bootstrap_servers=bootstrap_servers, value_serializer = lambda x : json.dumps(x).encode("utf-8"))
    for topic in topics:
        producer.send(topic, value=data.get(topic))
    producer.flush()

In [113]:
import uuid

def consumer_from_topics(topics, bootstrap_servers='localhost:9092'):
    # 고유한 컨슈머 그룹 ID 생성
    # group_id = f"consumer-group-{uuid.uuid4()}"
    
    consumer = KafkaConsumer(
        *topics,
        bootstrap_servers=bootstrap_servers,
        auto_offset_reset='earliest',
        # group_id=group_id,
        value_deserializer=lambda x: json.loads(x.decode('utf-8'))
    )

    for message in consumer:
        print(f"Topic: {message.topic}, Group_id: {group_id}, Partition: {message.partition}, Offset: {message.offset}, Key: {message.key}, Value: {message.value}")

In [114]:
file_path = 'data2.json'
topics = ['p1','p2','p3']
data = read_dta_from_file(file_path)

#토픽 생성
# create_topics(topics)
producer_to_topics(data, topics)
consumer_from_topics(topics)

Topic: p3, Group_id: test1, Partition: 0, Offset: 0, Key: None, Value: [{'id': 7, 'name': 'Grace'}, {'id': 8, 'name': 'Henry'}, {'id': 9, 'name': 'Ivy'}]
Topic: p3, Group_id: test1, Partition: 0, Offset: 1, Key: None, Value: [{'id': 7, 'name': 'Grace'}, {'id': 8, 'name': 'Henry'}, {'id': 9, 'name': 'Ivy'}]
Topic: p3, Group_id: test1, Partition: 0, Offset: 2, Key: None, Value: [{'id': 7, 'name': 'Grace'}, {'id': 8, 'name': 'Henry'}, {'id': 9, 'name': 'Ivy'}]
Topic: p3, Group_id: test1, Partition: 0, Offset: 3, Key: None, Value: [{'id': 7, 'name': 'Grace'}, {'id': 8, 'name': 'Henry'}, {'id': 9, 'name': 'Ivy'}]
Topic: p3, Group_id: test1, Partition: 0, Offset: 4, Key: None, Value: [{'id': 7, 'name': 'Grace'}, {'id': 8, 'name': 'Henry'}, {'id': 9, 'name': 'Ivy'}]
Topic: p3, Group_id: test1, Partition: 0, Offset: 5, Key: None, Value: [{'id': 7, 'name': 'Grace'}, {'id': 8, 'name': 'Henry'}, {'id': 9, 'name': 'Ivy'}]
Topic: p3, Group_id: test1, Partition: 0, Offset: 6, Key: None, Value: [{'id

KeyboardInterrupt: 

In [112]:
import threading

# 데이터와 토픽 읽기
file_path = 'data2.json'
topics = ['p1', 'p2', 'p3']
data = read_dta_from_file(file_path)

# 프로듀서 작업을 수행하는 함수
def run_producer(data, topics):
    # 여기서 토픽 생성이 필요하다면 주석을 해제하세요
    # create_topics(topics)
    producer_to_topics(data, topics)

# 컨슈머 작업을 수행하는 함수
def run_consumer(topics):
    consumer_from_topics(topics)

# 컨슈머 스레드 생성 및 시작
consumer_thread = threading.Thread(target=run_consumer, args=(topics,))
consumer_thread.start()

# 프로듀서 스레드 생성 및 시작
producer_thread = threading.Thread(target=run_producer, args=(data, topics))
producer_thread.start()

# 필요하다면 스레드들이 종료될 때까지 기다립니다.
producer_thread.join()
consumer_thread.join()

KeyboardInterrupt: 

Topic: p1, Group_id: test1, Partition: 0, Offset: 38, Key: None, Value: [{'id': 11, 'name': 'Alice1'}, {'id': 12, 'name': 'Bob2'}, {'id': 13, 'name': 'Charlie3'}]
Topic: p3, Group_id: test1, Partition: 0, Offset: 38, Key: None, Value: [{'id': 17, 'name': 'Grace1'}, {'id': 18, 'name': 'Henry1'}, {'id': 19, 'name': 'Ivy1'}]
Topic: p2, Group_id: test1, Partition: 0, Offset: 38, Key: None, Value: [{'id': 14, 'name': 'David1'}, {'id': 15, 'name': 'Emma1'}, {'id': 16, 'name': 'Frank1'}]


In [ ]:
data